In [9]:
date_str = '{0}{1:02d}{2:02d}'.format(2019, 7, 1)
url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'

query_params = {
    'date': date_str,
    'response': 'json',
    'type': 'ALL',
    '_': str(round(time.time() * 1000) - 500)
}

# Get json data
page = requests.get(url, params=query_params)
content = page.json()
content

{'data4': [['寶島股價報酬指數',
   '15,147.00',
   "<p style ='color:red'>+</p>",
   '243.89',
   '1.64'],
  ['發行量加權股價報酬指數',
   '19,650.99',
   "<p style ='color:red'>+</p>",
   '313.36',
   '1.62'],
  ['臺灣公司治理100報酬指數',
   '7,361.96',
   "<p style ='color:red'>+</p>",
   '117.19',
   '1.62'],
  ['臺灣50報酬指數', '14,925.47', "<p style ='color:red'>+</p>", '292.22', '2.00'],
  ['臺灣中型100報酬指數',
   '13,549.53',
   "<p style ='color:red'>+</p>",
   '269.01',
   '2.03'],
  ['臺灣資訊科技報酬指數', '18,330.54', "<p style ='color:red'>+</p>", '587.52', '3.31'],
  ['臺灣發達報酬指數', '13,963.97', "<p style ='color:red'>+</p>", '184.81', '1.34'],
  ['臺灣高股息報酬指數', '13,294.33', "<p style ='color:red'>+</p>", '217.63', '1.66'],
  ['臺灣就業99報酬指數', '8,555.22', "<p style ='color:red'>+</p>", '124.45', '1.48'],
  ['臺灣高薪100報酬指數', '7,352.56', "<p style ='color:red'>+</p>", '89.82', '1.24'],
  ['未含金融電子報酬指數', '23,642.79', "<p style ='color:red'>+</p>", '41.91', '0.18'],
  ['小型股300報酬指數', '7,389.33', "<p style ='color:red'>+</p>", '81.66', 

In [13]:
#date_str = '{0}/{1:02d}/{2:02d}'.format(108,7,1)
#ttime = str(int(time.time()*100))
#url = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&d={}&_={}'.format(date_str, ttime)
#page = requests.get(url)
#result = page.json()
result

{'reportDate': '108/07/01',
 'reportTitle': '上櫃股票行情(含等價、零股、盤後、鉅額交易)',
 'iTotalRecords': 4490,
 'iTotalDisplayRecords': 4490,
 'listNum': '776',
 'totalAmount': '32,957,628,878',
 'totalVolumn': '401,089,910',
 'totalCount': '235,126',
 'mmData': [['4415',
   '台原藥',
   '6.00',
   '0.00 ',
   '6.00',
   '6.00',
   '6.00',
   '6.00',
   '1,005',
   '6,030',
   '6',
   '6.00',
   '6.56',
   '43,577,168',
   '6.00',
   '6.60',
   '5.40']],
 'aaData': [['006201',
   '元大富櫃50',
   '12.69',
   '+0.46',
   '12.34',
   '12.71',
   '12.34',
   '12.57',
   '280,244',
   '3,523,316',
   '140',
   '12.69',
   '12.70',
   '42,946,000',
   '12.69',
   '13.95',
   '11.43'],
  ['00679B',
   '元大美債20年',
   '42.12',
   '-0.26 ',
   '42.38',
   '42.38',
   '42.08',
   '42.13',
   '1,255,597',
   '52,904,261',
   '309',
   '42.11',
   '42.12',
   '638,192,000',
   '42.12',
   '9999.95',
   '0.01'],
  ['00687B',
   '國泰20年美債',
   '43.94',
   '-0.28 ',
   '44.01',
   '44.01',
   '43.94',
   '43.95',
   '270,000'

In [4]:
# -*- coding: utf-8 -*-

import os
import re
import sys
import csv
import time
import string
import logging
import requests
import argparse
from datetime import datetime, timedelta
import pandas as pd
from os import mkdir
from os.path import isdir

class Crawler():
    def __init__(self, prefix="data"):
        ''' Make directory if not exist when initialize '''
        if not isdir(prefix):
            mkdir(prefix)
        self.prefix = prefix

    def _clean_row(self, row):
        ''' Clean comma and spaces '''
        for index, content in enumerate(row):
            row[index] = re.sub(",", "", content.strip())
        return row

    def _record(self, stock_id, row):
        ''' Save row to csv file '''
        f = open('{}/{}.csv'.format(self.prefix, stock_id), 'a')
        cw = csv.writer(f, lineterminator='\n')
        cw.writerow(row)
        f.close()

    def _get_tse_data(self, date_tuple):
        date_str = '{0}{1:02d}{2:02d}'.format(date_tuple[0], date_tuple[1], date_tuple[2])
        url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'

        query_params = {
            'date': date_str,
            'response': 'json',
            'type': 'ALL',
            '_': str(round(time.time() * 1000) - 500)
        }

        # Get json data
        page = requests.get(url, params=query_params)

        if not page.ok:
            logging.error("Can not get TSE data at {}".format(date_str))
            return

        content = page.json()

        # For compatible with original data
        date_str_mingguo = '{0}/{1:02d}/{2:02d}'.format(date_tuple[0] - 1911, date_tuple[1], date_tuple[2])

        for data in content['data9']:
            sign = '-' if data[9].find('green') > 0 else ''
            row = self._clean_row([
                date_str_mingguo, # 日期
                data[2], # 成交股數
                data[4], # 成交金額
                data[5], # 開盤價
                data[6], # 最高價
                data[7], # 最低價
                data[8], # 收盤價
                sign + data[10], # 漲跌價差
                data[3], # 成交筆數
            ])

            self._record(data[0].strip(), row)

    def _get_otc_data(self, date_tuple):
        date_str = '{0}/{1:02d}/{2:02d}'.format(date_tuple[0] - 1911, date_tuple[1], date_tuple[2])
        ttime = str(int(time.time()*100))
        url = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&d={}&_={}'.format(date_str, ttime)
        page = requests.get(url)

        if not page.ok:
            logging.error("Can not get OTC data at {}".format(date_str))
            return

        result = page.json()

        if result['reportDate'] != date_str:
            logging.error("Get error date OTC data at {}".format(date_str))
            return

        for table in [result['mmData'], result['aaData']]:
            for tr in table:
                row = self._clean_row([
                    date_str,
                    tr[8], # 成交股數
                    tr[9], # 成交金額
                    tr[4], # 開盤價
                    tr[5], # 最高價
                    tr[6], # 最低價
                    tr[2], # 收盤價
                    tr[3], # 漲跌價差
                    tr[10] # 成交筆數
                ])
                self._record(tr[0], row)


    def get_data(self, date_tuple):
        print('Crawling {}'.format(date_tuple))
        self._get_tse_data(date_tuple)
        self._get_otc_data(date_tuple)

In [67]:
if not os.path.isdir('log'):
    os.makedirs('log')
logging.basicConfig(filename='log/crawl-error.log',
    level=logging.ERROR,
    format='%(asctime)s\t[%(levelname)s]\t%(message)s',
    datefmt='%Y/%m/%d %H:%M:%S')

# Get arguments
parser = argparse.ArgumentParser(description='Crawl data at assigned day')
parser.add_argument('day', type=int, nargs='*',
    help='assigned day (format: YYYY MM DD), default is today')
parser.add_argument('-b', '--back', action='store_true',
    help='crawl back from assigned day until 2004/2/11')
parser.add_argument('-c', '--check', action='store_true',
    help='crawl back 10 days for check data')

_StoreTrueAction(option_strings=['-c', '--check'], dest='check', nargs=0, const=True, default=False, type=None, choices=None, help='crawl back 10 days for check data', metavar=None)

In [73]:
first_day = datetime.today()
last_day = datetime(2004, 2, 11)
crawler = Crawler()
max_error = 5
error_times = 0
while error_times < max_error and first_day >= last_day:
    try:
        crawler.get_data((first_day.year, first_day.month, first_day.day))
        error_times = 0
    except:
        date_str = first_day.strftime('%Y/%m/%d')
        logging.error('Crawl raise error {}'.format(date_str))
        error_times += 1
        continue
    finally:
        first_day -= timedelta(1)

Crawling (2019, 9, 19)
Crawling (2019, 9, 18)
Crawling (2019, 9, 17)
Crawling (2019, 9, 16)
Crawling (2019, 9, 15)


ERROR:root:Crawl raise error 2019/09/15


Crawling (2019, 9, 14)


ERROR:root:Crawl raise error 2019/09/14


Crawling (2019, 9, 13)


...

ERROR:root:Crawl raise error 2019/02/08


Crawling (2019, 2, 7)


ERROR:root:Crawl raise error 2019/02/07


Crawling (2019, 2, 6)


ERROR:root:Crawl raise error 2019/02/06


In [18]:
FOLDER = 'data'
file_name = "9110.csv"
dict_rows = {}

# Load and remove duplicates (use newer)
with open('{}/{}'.format(FOLDER, file_name), 'r') as file:
    for line in file.readlines():
        dict_rows[line.split(',', 1)[0]] = line

dict_rows['108/02/11']

<class 'str'>


In [101]:
duration_covered = pd.read_csv('duration_coverage.csv') 
duration_covered

,Created_at,Date
0,2019-09-20 23:46:42.918175,83/09/05
1,2019-09-20 23:46:42.922874,83/09/06
2,2019-09-20 23:46:42.930208,83/09/07
3,2019-09-20 23:46:42.936231,83/09/08
4,2019-09-20 23:46:42.940320,83/09/09
5,2019-09-20 23:46:42.944173,83/09/10
6,2019-09-20 23:46:42.947211,83/09/12
7,2019-09-20 23:46:42.950897,83/09/13
8,2019-09-20 23:46:42.958859,83/09/14
9,2019-09-20 23:46:42.963615,83/09/15


In [93]:
TSMC = pd.read_csv('data/2330.csv', header=None) 
existed_day = TSMC.iloc[:,0].tolist()
for i in existed_day:
    duration_covered = duration_covered.append(pd.DataFrame({'Date':[str(i)], 'Created_at':[datetime.now()]}), sort=True)

In [111]:
existed_day = duration_covered['Date'].tolist()
handling_date = str(first_day.year-1911)+str(first_day.strftime("/%m/%d"))

handling_date

'108/09/20'